### this script distribute funded publications to funder country by fractional counting. The weight to each fractional counting to funder country is proportional to the number of funding instances (FA-GN entry). For example, 2 funding instance from China and 1 from US, then China funds 2/3 of the pub and US funds 1/3

The input of the file is the publication data

The output of the file is fractional funding data includes:
- funder: the funding country
- year
- intcol: whether the amount of funding is contributed to international coauthored publications or not.
- cnt: the number of funded publications. 

In [1]:
import pandas as pd
import numpy as np
from itertools import product

# Parameters

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pub.pkl"

cntry_fund_frac_path='../../data/nf_folder/Data/DerivedData/Derived/cntry_fund_frac.csv'

In [3]:
# Parameters
pubs_path = "../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"
cntry_fund_frac_path = (
    "../data/NationalFunding/Data/DerivedData/Derived/cntry_fund_frac.csv"
)


In [4]:
pubs_df=pd.read_pickle(pubs_path)
pubs_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[Chile, France]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


# assign each paper to its funding country by fractional counting

In [5]:
cntry_fund_frac = []
for index, row in pubs_df.iterrows():
    year=row['year']
    coll=row['IntCol']
    funder_cntrys=row['funder']
    if funder_cntrys=='Not-Funded':
        cntry_fund_frac.append([funder_cntrys,1,year,coll])
    else:
        n=len(funder_cntrys)
        for c in funder_cntrys:
            cntry_fund_frac.append([c,1/n,year,coll])

In [6]:
frac_df = pd.DataFrame(cntry_fund_frac,
                                  columns=['funder','cnt','year','intcol'])

In [7]:
frac_df=frac_df.groupby(['funder','year','intcol'])['cnt'].sum().reset_index()

In [8]:
frac_df.head()

,funder,year,intcol,cnt
0,Afghanistan,2010,1,1.000000
1,Afghanistan,2011,1,0.333333
2,Afghanistan,2012,1,0.333333
3,Afghanistan,2015,0,0.148352
4,Afghanistan,2015,1,0.279167


In [9]:
frac_df.to_csv(cntry_fund_frac_path,index=False)